In [ ]:
from openai import OpenAI
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
import whisper
import json

In [ ]:
try:
    model = whisper.load_model("medium")
    result = model.transcribe(r"audio.mp3")

    with open(r"content\test.txt","w") as file:
        file.write(result['text'])
        
except Exception as error:
    print(error)

In [ ]:
# Não deu pra integrar com o GPT porque tem que pagar ;-;
# client = OpenAI(api_key = 'api-key')

# def get_completion(prompt, model="gpt-3.5-turbo"):
#     message = [{"role": "user", "content": prompt}]
#     response = client.chat.completions.create(model=model,messages=message)
#     return response.choices[0].message["content"]

# prompt = f"Faça um resumo de 20 linhas e corrija todos os � que tem no texto a seguir {result['text']}"
# response = get_completion(prompt)
# print(response)

In [ ]:
def add_to_textfile(text_file, text):
  text = text.encode('utf-8')
  with open(text_file, 'w', encoding='utf-8') as f:
    f.write(text.decode('utf-8'))
  return text.decode('utf-8')  

texto = add_to_textfile(r"content\test.txt",result['text'])
print(texto)

In [ ]:
parser = PlaintextParser.from_string(texto,Tokenizer('portuguese'))
sumarizer = LuhnSummarizer()
summary = sumarizer(parser.document,40)
summary